In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install and import library

In [ ]:
!pip install tiktoken
!pip install evaluate
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from transformers import GPT2Model
import torch.nn as nn
import tiktoken
import os
import urllib.request
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Preprocessing

In [ ]:
def preprocess(file_csv, tokenizer):
    df = pd.read_csv(file_csv)
    datasets = []

    ingre = [' '.join(text for text in json.loads(df['ingredients'][i])) for i in range(len(df['ingredients']))]
    direc = [' '.join(text for text in json.loads(df['directions'][i])) for i in range(len(df['directions']))]

    texts = [ingre[i] + '\nIntructions: ' + direc[i] for i in range(len(ingre))]

    [datasets.append(torch.tensor(tokenizer.encode(text))) for text in texts]

    return datasets


def pad_and_truncate_sequences(sequences, max_length, padding_value=50256):
    padded_sequences = []
    for seq in sequences:
        if len(seq) < max_length:
            padding = [padding_value] * (max_length - len(seq))
            padded_seq = torch.cat((seq, torch.tensor(padding)))
            padded_sequences.append(padded_seq)
        else:

            padded_sequences.append(seq[:max_length])

    padded_sequences = torch.stack(padded_sequences)

    return padded_sequences

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())

### Multi-head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, block_size, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        # Reduce the projection dim to match desired output dim
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Linear layer to combine head outputs
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(
            torch.ones(block_size, block_size), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        # Dot product for each head
        attn_scores = queries @ keys.transpose(2, 3)

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

### Layer normalize + Feed forward (MLP)

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
            nn.Dropout(cfg["drop_rate"])
        )

    def forward(self, x):
        return self.layers(x)

### GPT2 Block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            block_size=cfg["seq_len"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_resid(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut  # Add the original input back

        return x

### GPT2 Model

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["seq_len"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

### Load weights

In [ ]:
def assign_check(left, right):
    if left.shape != right.shape:
        raise ValueError(
            f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

import numpy as np


def load_weights(gpt, gpt_hf):

    d = gpt_hf.state_dict()

    gpt.pos_emb.weight = assign_check(gpt.pos_emb.weight, d["wpe.weight"])
    gpt.tok_emb.weight = assign_check(gpt.tok_emb.weight, d["wte.weight"])

    for b in range(BASE_CONFIG["n_layers"]):
        q_w, k_w, v_w = np.split(d[f"h.{b}.attn.c_attn.weight"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign_check(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign_check(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign_check(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(d[f"h.{b}.attn.c_attn.bias"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign_check(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign_check(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign_check(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign_check(
            gpt.trf_blocks[b].att.out_proj.weight, d[f"h.{b}.attn.c_proj.weight"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign_check(
            gpt.trf_blocks[b].att.out_proj.bias, d[f"h.{b}.attn.c_proj.bias"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign_check(
            gpt.trf_blocks[b].ff.layers[0].weight, d[f"h.{b}.mlp.c_fc.weight"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign_check(
            gpt.trf_blocks[b].ff.layers[0].bias, d[f"h.{b}.mlp.c_fc.bias"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign_check(
            gpt.trf_blocks[b].ff.layers[2].weight, d[f"h.{b}.mlp.c_proj.weight"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign_check(
            gpt.trf_blocks[b].ff.layers[2].bias, d[f"h.{b}.mlp.c_proj.bias"])

        gpt.trf_blocks[b].norm1.scale = assign_check(
            gpt.trf_blocks[b].norm1.scale, d[f"h.{b}.ln_1.weight"])
        gpt.trf_blocks[b].norm1.shift = assign_check(
            gpt.trf_blocks[b].norm1.shift, d[f"h.{b}.ln_1.bias"])
        gpt.trf_blocks[b].norm2.scale = assign_check(
            gpt.trf_blocks[b].norm2.scale, d[f"h.{b}.ln_2.weight"])
        gpt.trf_blocks[b].norm2.shift = assign_check(
            gpt.trf_blocks[b].norm2.shift, d[f"h.{b}.ln_2.bias"])

        gpt.final_norm.scale = assign_check(
            gpt.final_norm.scale, d[f"ln_f.weight"])
        gpt.final_norm.shift = assign_check(
            gpt.final_norm.shift, d[f"ln_f.bias"])
        gpt.out_head.weight = assign_check(
            gpt.out_head.weight, d["wte.weight"])

### Initialize model

Load openAI's GPT2 model

In [ ]:
model_names = {
    "gpt2-small": "openai-community/gpt2",         # 124M
    "gpt2-medium": "openai-community/gpt2-medium",  # 355M
    "gpt2-large": "openai-community/gpt2-large",   # 774M
    "gpt2-xl": "openai-community/gpt2-xl"          # 1558M
}

CHOOSE_MODEL = "gpt2-small"

gpt_hf = GPT2Model.from_pretrained(model_names[CHOOSE_MODEL], cache_dir="checkpoints")
gpt_hf.eval()

BASE_CONFIG = {
    "vocab_size": 50257,  # Vocabulary size
    "seq_len": 1024,      # Context length
    "drop_rate": 0.1,     # Dropout rate
    "qkv_bias": True      # Query-key-value bias
}

model_configs = {
    "gpt2-small": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# text_test = "1 c. firmly packed brown sugar 1/2 c. evaporated milk 1/2 tsp. vanilla 1/2 c. broken nuts (pecans) 2 Tbsp. butter or margarine 3 1/2 c. bite size shredded rice biscuits\nInstruction: "

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpt2_model = GPTModel(BASE_CONFIG)
load_weights(gpt2_model, gpt_hf)

gpt2_model.eval()
gpt2_model.to(device)

<ipython-input-20-67f9f4c1d4b2>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.nn.Parameter(torch.tensor(right))


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttentio

Load my model

In [ ]:
model_names = {
    "gpt2-small": "openai-community/gpt2",         # 124M
    "gpt2-medium": "openai-community/gpt2-medium",  # 355M
    "gpt2-large": "openai-community/gpt2-large",   # 774M
    "gpt2-xl": "openai-community/gpt2-xl"          # 1558M
}

CHOOSE_MODEL = "gpt2-small"

BASE_CONFIG = {
    "vocab_size": 50257,  # Vocabulary size
    "seq_len": 512,      # Context length
    "drop_rate": 0.1,     # Dropout rate
    "qkv_bias": True      # Query-key-value bias
}

model_configs = {
    "gpt2-small": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

my_model = GPTModel(BASE_CONFIG)

weights_path = "/content/drive/MyDrive/nothing/weights_model-gpt-small-best.pth"
my_model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))

my_model.eval()
my_model.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(512, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention

### Generate text

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temperature, top_k=None):
    idx = idx.to(device)

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(
                float('-inf')).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(
                probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(
                logits, dim=-1, keepdim=True)  # (batch_size, 1)

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx


def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [ ]:
file_csv = '/content/RecipeNLG_dataset_evaluate.csv'
tokenizer = tiktoken.get_encoding('gpt2')
max_length = 1025

df = pd.read_csv(file_csv)
val_input = []

ingre = [' '.join(text for text in json.loads(df['ingredients'][i])) for i in range(len(df['ingredients']))]
direc = [' '.join(text for text in json.loads(df['directions'][i])) for i in range(len(df['directions']))]

texts = [ingre[i] + '\nIntructions: ' for i in range(len(ingre))]

[val_input.append(torch.tensor(tokenizer.encode(text)).to(device)) for text in texts]

val_input = val_input[:20]

print(val_input[0])

tensor([ 1065, 35104,  5937,    11,   319,   262,  7582,    11, 37222,  2945,
          718, 24314, 10360,   979,   315,  1462,    11, 10284,  1079,  4129,
         1322,   718, 47234, 16597,   728,  1010, 10695, 10746,   513, 30064,
          308,  3447,   306,  1036,   515,  1582,  6880,   272,  9891,   352,
        23972,    11, 42617,   290,   390,    12,   325, 15395,    11, 48898,
         3734,   352, 18873,  4713,  3013,  3949,   442,  1083,   357,  1462,
        36167,   680,     8,   198,  5317,  1356,   507,    25,   220],
       device='cuda:0')


In [ ]:
def remove_suffix(s):
    return s[:s.find('<')]

In [ ]:
id = 0
gpt2_result = []
for item in val_input:
    print(id, end='\t')
    id += 1
    item = torch.unsqueeze(item, 0)
    gpt2_output = generate(
        model=gpt2_model,
        idx=item,
        max_new_tokens=256,
        context_size=BASE_CONFIG["seq_len"],
        top_k=5,
        temperature=0.8
    )
    gpt2_result.append(gpt2_output.to('cpu'))

# print(token_ids_to_text(result[0], tokenizer))

0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	

In [ ]:
for item in gpt2_result:
    print(remove_suffix(token_ids_to_text(item, tokenizer)) + '\n===========================\n')

12 oysters, on the shell, loosened 6 slices prosciutto, halved lengthways 6 teaspoons Worcestershire sauce 3 tablespoons greshly grated parmesan cheese 1 tomatoes, peeled and de-seeded, diced fine 1 lemon fresh snipped chives (to garnish)
Intructions:  1. Preheat the oven to 425 degrees Fahrenheit. In a bowl, mix together the oysters, parsley and parmesan.
2. Mix together the salt and pepper.
3. Pour the mixture into a large mixing bowl.
4. Mix in the oysters, parsley and parmesan. Mix in the lemon juice and the salt.
5. Mix in the garlic, ginger, and garlic powder. Mix well.
6. Pour the mixture into a small bowl and stir well.
7. Serve warm.
This is a recipe for a delicious and delicious oyster.
This recipe is from a cookbook I read in the early 2000's.
I have a few other recipes I've read about.
Enjoy!
You can find all my recipes in my cookbook, and if you want to see more, check out my cook book, The Cookbook of Oysters.
If you have any questions, comments, suggestions, or just want

In [ ]:
id = 0
my_result = []
for item in val_input:
    print(id, end='\t')
    id += 1
    item = torch.unsqueeze(item, 0)
    my_model_output = generate(
        model=my_model,
        idx=item,
        max_new_tokens=256,
        context_size=BASE_CONFIG["seq_len"],
        top_k=5,
        temperature=0.8
    )
    my_result.append(my_model_output.to('cpu'))

# print(token_ids_to_text(result[0], tokenizer))

0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	

In [ ]:
for item in my_result:
    print(remove_suffix(token_ids_to_text(item, tokenizer)) + '\n===========================\n')

12 oysters, on the shell, loosened 6 slices prosciutto, halved lengthways 6 teaspoons Worcestershire sauce 3 tablespoons greshly grated parmesan cheese 1 tomatoes, peeled and de-seeded, diced fine 1 lemon fresh snipped chives (to garnish)
Intructions:  the split peas. Stuff with waxed paper and allow to 2 cups of green beans plus 15 minutes.

1 lb lean ground beef 2 cups Italian-style tomato sauce 2 -10 inches prepared pizza crust, prebaked 1/2 cup green onion, chopped 1 small green pepper, seeded, chopped 1 small yellow onion, diced 1 cup mozzarella cheese, shredded 1 teaspoon vegetable oil 1/2 teaspoon salt 1/2 teaspoon pepper
Intructions:  both 4 ingredients except green pepper. Stir everything preference over meat. Pour into greased casserole and cheese over top. Bake at 375° for about 45 minutes.

3 tablespoons lime juice 2 teaspoons olive oil 1 teaspoon hot pepper flakes 2 cups finely shredded green cabbage 1 cup finely shredded red cabbage
Intructions:  Mist to mix well.

2 cups

In [ ]:
print(*direc, sep='\n\n')

Remove oyster from shell, wrap in a half slice of prosciutto; return to shell. Top each oyster with half a teaspoon of Worcestershire sauce and a teaspoon of Parmesan. Place under the griller (broiler) and cook for 2-3 minutes or until cheese has browned a little and prosciutto and oyster just heated through. Remove from the griller (broiler) and top the hot oysters with a teaspoon of diced tomato, a squeeze of lemon juice and a sprinkling of snipped chives. SERVE AT ONCE (that's an instruction for Peter J who says many of JanRoundOz's recipes finish that way ;-).

Preheat oven to 400. In large skillet, heat oil over medium heat, add beef and cook, breaking up, for 5-10 minutes. Until no longer pink. Drain off fat. Stir in yellow onion and peppers. cook 5 minutes. Stir in tomato sauce, salt and pepper. Simmer for 5 minutes. Place 1 pizza crust on baking sheet, spread half of the meat sauce over crust. sprinkle 1/2 of the green onion and 1/2 the cheese over top. Bake for 20 - 25 mins, o

### output

In [ ]:
my_model_output = """12 oysters, on the shell, loosened 6 slices prosciutto, halved lengthways 6 teaspoons Worcestershire sauce 3 tablespoons greshly grated parmesan cheese 1 tomatoes, peeled and de-seeded, diced fine 1 lemon fresh snipped chives (to garnish)
Intructions:  buy 9 to 14 minutes per pound nonstick pan with foil (8 to 15 minutes).
===========================

1 lb lean ground beef 2 cups Italian-style tomato sauce 2 -10 inches prepared pizza crust, prebaked 1/2 cup green onion, chopped 1 small green pepper, seeded, chopped 1 small yellow onion, diced 1 cup mozzarella cheese, shredded 1 teaspoon vegetable oil 1/2 teaspoon salt 1/2 teaspoon pepper
Intructions:  the beef with a slice of green pepper, onion rings and rice. Cut up in strips and then marinate in a quart or large skillet. Remove meat from oven; drain off fat. Add all ingredients to spaghetti. Cook at once. Pour into a 1/2 quart ungreased casserole. Top with cheese. As soup and vegetables, scum fashion on top ring. Bake for 30 to 40 minutes, covered and 15 minutes. Serves 8.
===========================

3 tablespoons lime juice 2 teaspoons olive oil 1 teaspoon hot pepper flakes 2 cups finely shredded green cabbage 1 cup finely shredded red cabbage
Intructions:  Mist to line a 9 x 12-inch pan. Mix ingredients together in order listed. Pour over and let stand in the refrigerator overnight.
===========================

2 cups cooked spinach 1 cup ricotta cheese 2 eggs 1/2 cup sun-dried tomato, chopped (optional) 1/4 cup shredded fresh basil leaves or 1 tablespoon dried basil 1 tablespoon olive oil salt and pepper 1 partially 9-inch baked pie crust
Intructions: -fry or pulp. Remove skin from food processor and pecans. Mix eggs, milk and salt in food processor and pepper. Next, food coloring. Pour over graham cracker crust. Sprinkle with cheese. Bake at 350° for 50 minutes to one hour until berries are done.
===========================

2 cups shredded monterey jack cheese 1 (8 ounce) package cream cheese, softened 1 cup sour cream 1/2 cup mayonnaise 1/4 cup minced red bell pepper 1/4 cup minced green onion 2 tablespoons chopped fresh parsley 2 tablespoons fresh lemon juice 1 1/2 teaspoons Old Bay Seasoning 1/4 teaspoon ground red pepper 2 (8 ounce) containers lump crabmeat, picked 1/4 cup freshly grated parmesan cheese toasted french bread rounds or cracker
Intructions: arsely dice paper thin slices of spaghetti sauce, cream cheese and sour cream together until smooth. Add drained tomatoes, crabmeat, and Parmesan cheese. Beat until piping stiff. Drain well. If mixture is available, mix corn, broccoli and 1/2 tablespoons cottage cheese. Sprinkle with parsley flakes on top. Bake at 350° for 30 minutes.
===========================

1 tablespoon butter (softened) 1 cup milk 1/4 cup heavy cream 2/3 cup flour 3 eggs 1/4 cup sugar 2 teaspoons vanilla 1 teaspoon lemon zest 1/2 teaspoon cinnamon 1/4 teaspoon salt 4 cups apples (tart-sliced) 1 tablespoon powdered sugar
Intructions:  butter in blender until light and fluffy. Beat in remaining ingredients. Beat in mixer at high speed for 1 hour. Stir in vanilla. Beat egg whites. Pour into well-greased and bake at 325° for about 50 minutes. Place pie shell on top of foil and bake 25 to 30 minutes longer at 350°. Cool one minute. Serve with whipped cream. Makes 8.
===========================

6 quarts figs baking soda (one cup) 1 gallon water, boiling cold water 1 lemon, sliced (optional) 1 orange, sliced (optional) lemon juice (use the bottled kind) Syrup 2 1/4 cups sugar 5 1/4 cups water
Intructions: ? Mix together yogurt, water, and sugar in food bowl. Add lemon juice gradually. Pour over crackers; let stand in refrigerator overnight tossing until to serve.
===========================

1 cup vegetable stock 3 cups chopped broccoli (about 1/2 large bunch) 1 garlic clove, minced 2 teaspoons grated gingerroot 1 cup soymilk or 1 cup skim milk 1 teaspoon low sodium soy sauce 1/4 cup finely chopped red onion
Intructions:  vegetables lengthwise and mix well. Saute chopped chafers 2 tablespoons butter. Add onion and pepper, then add rice which has been cooked and drained.
===========================

2 lbs chicken (bone-in or boneless, your choice) 1 cup Bourbon 1 cup soy sauce 1/2 cup brown sugar 1 teaspoon paprika 1 tablespoon molasses 3 garlic cloves, smashed 2 tablespoons ginger, freshly grated 1 1/2 teaspoons onion powder 1 1/2 teaspoons garlic powder
Intructions:  broth chicken in small bowl. Add remaining ingredients; bring to boiling. Reduce heat down and cook 30 minutes longer until chicken is done.
===========================

FOR THE CHICKEN 1 (3 -3 1/2 lb) roasting chickens 2 ounces fresh gingerroot, crushed 2 garlic cloves, crushed 1 scallion, tie in a know 1 teaspoon salt 2 tablespoons vegetable oil or 2 tablespoons peanut oil chili sauce or soy sauce, for dipping FOR THE RICE 2 tablespoons vegetable oil or 2 tablespoons peanut oil 5 garlic cloves, finely chopped 5 shallots, finely chopped 1 1/2 cups long grain rice 3 3/4 cups chicken stock 1 teaspoon salt
Intructions:  the potatoes (rinse with cold water to the directions on both sides of a very warm. While cool, add eggs, one at a time to the time. They are now ready to be made a stiff. Keep the pancakes, as well as a few days in center of a crock-pot the rest of the oven.
===========================

1 cup honey, boiled and brought to room temperature 1/2 cup butter 1 cup brown sugar 4 eggs, separated 3 cups all-purpose flour 2 teaspoons baking soda 1/2 teaspoon baking powder 1 teaspoon cinnamon 1/4 teaspoon salt 1 cup sour cream 1 cup walnuts, chopped
Intructions:  Mix all ingredients in bowl and beat. Pour into greased and floured 7 x 11-inch pan. Bake at 350° for 30 to 35 minutes.
===========================

3 tablespoons brown sugar 3 tablespoons white vinegar 1 onion, peeled and cut into chunks 2 garlic cloves, peeled and lightly crushed 1 teaspoon peppercorn, lightly crushed 2 slices orange peel (about 2 inches each) 1 star anise 1 1/2 kg pickled pork
Intructions: ? Let marinate in beer overnight.
===========================

2 cups cooked rice 2 cups canned corn or 2 cups frozen corn 16 ounces joan of arc spicy hot chili beans, drained 1 (10 ounce) can rotel 4 ounces Velveeta reduced fat cheese product
Intructions:  cans of corn, rice and cut in small pieces. Add soups and juice. Stir to mix well. Lay chicken in bottom of 8 x 5-inch dish. Pour remaining ingredients over top of rice. Cover and chill at least 45 minutes.
===========================

Cake 1/2 cup cocoa powder, such as valrhona 1 cup warm water 1/2 cup plus 2 tablespoons vegetable oil 1 1/4 cups plus 2 tablespoons granulated sugar 2 large eggs 2 teaspoons vanilla extract 1 1/4 cups plus 2 tablespoons all-purpose flour 1 1/2 teaspoons baking soda 1 teaspoon kosher salt 3/4 cup plus 2 tablespoons buttermilk Caramel Sauce 1 cup granulated sugar 1 cup heavy cream 1 tablespoon salt Buttercream 8 whole egg whites 2 cups granulated sugar 2 lbs unsalted butter caramel sauce (from above) Eggnog Cheesecake 1 lb cream cheese 3/4 cup granulated sugar 2 large whole eggs plus 1 egg yolks 1/4 cup creme fraiche 1 teaspoon vanilla extract 1 tablespoon Irish whiskey, such as 2 gingers 2 teaspoons freshly grated nutmeg 1 teaspoon kosher salt Toffee Pecan Pie 9 inches pie crusts (homemade or store bought) 5 tablespoons unsalted butter 1 cup packed dark brown sugar 3 large eggs 2 teaspoons all-purpose flour 3/4 cup dark corn syrup 1 1/2 teaspoons kosher salt 1 tablespoon vanilla extract 1 1/2 toasted pecan pieces 3/4 cup chopped Heath candy bar, pieces Assembly 1/2 cup cocoa powder, such as valrhona 1 cup chopped Heath candy bar, pieces chocolate, santas chocolate gold coins, white sprinkles, gold leaf or marzipan, woodland creatures
Intructions:  to taste:
Intructions: Follow:
Intructions: Mix well. Combine first 5 ingredients. Place into pie crust in bowl. Line 13 with graham cracker. Mix thoroughly. Beat 2 tablespoons. Pour into crusts and butter. Beat 2 minutes. Mix 2 tablespoons sugar, food processor at medium speed until well after crust. Stir in fruit and nuts are crumbiflower and nuts. Pour into crust. Pour into prepared crust. Mix remaining crumb mixture, pineapple, pineapple, butter and butter. Pour cooled fruit and nuts. Pour into pie crust. Bake at 350° for 25 to 30 minutes. Cool in pan or until a toothpick comes out. Cool in center comes x 2 hours.
===========================

1 apple 1/4 cup cottage cheese, whipped 1 egg cinnamon and artificial sweetener, to taste
Intructions: agne oysters and cut into pieces. Add lemon juice and spices. Heat until well thickens. Fold in whipped topping. Spoon into graham cracker pie crust. Bake at 350° for 45 minutes.
===========================

2 lbs carrots 2 medium onions 1 teaspoon pepper 1/2 teaspoon sugar 1/2 teaspoon Accent seasoning 8 slices bacon
Intructions:  white beans and remove seeds. Boil noodles until tender. In a large pot, saute onions in bacon. Add broth and remaining ingredients. Cook until thick. Can add bacon to skillet if desired thickness.
===========================

DRESSING 1 tablespoon minced ginger 1 garlic clove, minced 2 teaspoons light soy sauce 1/4 cup rice wine vinegar 2 oranges, juice of 1 lime, juice of 1 teaspoon brown sugar SALAD 2 cups Chinese cabbage, shredded 100 g bean sprouts 1 red onion, thinly sliced 1 orange, peeled and sliced into 1 inch segments 2 grilled chicken breasts, sliced chopped fresh cilantro (for garnish) salt & freshly ground black pepper 1/4 cup fresh cilantro, chopped
Intructions: : Coarsely dice well before removing vegetables, limes and place in a large frying pan. Add water, thyme, soy sauce and green pepper. Heat first to 1/4 cup water. Add more water; stir until rice. Prepare egg yol; mix well. Pour over cooked chicken in large bowl. Saute until bubbles; cook, about 30 minutes.
===========================

1 lb ground meat 2 garlic cloves, minced 3 ounces shallots 1 (24 ounce) jar premium cheese flavor spaghetti sauce 1 (14 1/2 ounce) can Italian stewed tomatoes 1 tablespoon oregano 1 teaspoon salt
Intructions:  chilies in shallow pan. Add 1 small chopped onion and garlic. Combine all ingredients except spaghetti and cover. Pour into crock-pot. Cook on high for 1 hour at 325°.
===========================

1/2 cup olive oil 2 ounces parmesan cheese 2 garlic cloves, peeled 3 tablespoons red wine vinegar 1 tablespoon mayonnaise 3 tablespoons sour cream 1/2 teaspoon italian seasoning 3 basil leaves, fresh salt, to taste pepper, to taste
Intructions:  10 ingredients. Pour into empty well with the dip to the malted water and let set in the refrigerator for at least 1 hour before serving.
===========================

1/4 cup whole almond, plus 2 tablespoons sliced almonds 1/4 teaspoon ground cinnamon 1/4 teaspoon freshly ground nutmeg 1 tablespoon trans-fat free margarine 2 cups sliced strawberries 1 cup blueberries 1 tablespoon granular sugar substitute 6 tablespoons part-skim ricotta cheese
Intructions:  bananas in a food processor or blender, cinnamon and all ingredients until well blended. Coarse whipping, making sure just before serving.
==========================="""

In [ ]:
gpt2_output = """12 oysters, on the shell, loosened 6 slices prosciutto, halved lengthways 6 teaspoons Worcestershire sauce 3 tablespoons greshly grated parmesan cheese 1 tomatoes, peeled and de-seeded, diced fine 1 lemon fresh snipped chives (to garnish)
Intructions:  Mix together the egg whites, salt, and pepper in a bowl. Add the garlic and cook until the onion and tomato have softened, about 3 minutes. Add the tomatoes, the salt, and the parmesan. Cook for about 1 minute, stirring frequently. Stir in the lemon juice until it thickens. Stir in the parsley and the cumin. Add the tomatoes, and cook for about 2 minutes, stirring frequently. Add the lemon juice and cook until the parsley has softened. Stir in the parsley, the garlic, the parmesan, and the cumin. Add the tomatoes, and cook until the parsley has softened. Stir in the cumin. Add the garlic, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened
===========================

1 lb lean ground beef 2 cups Italian-style tomato sauce 2 -10 inches prepared pizza crust, prebaked 1/2 cup green onion, chopped 1 small green pepper, seeded, chopped 1 small yellow onion, diced 1 cup mozzarella cheese, shredded 1 teaspoon vegetable oil 1/2 teaspoon salt 1/2 teaspoon pepper
Intructions:  Preheat the oven to 375 degrees F.
In a large bowl, whisk together the milk, tomato sauce, onion and peppers.
In a small bowl, mix together the dry ingredients, the dry ingredients, salt and pepper.
In a large bowl, mix together the tomato sauce, onion and peppers.
In a small bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
In a large bowl, mix together the dry ingredients, salt and pepper.
The final result should be a creamy cheese that is very moist and creamy
===========================

3 tablespoons lime juice 2 teaspoons olive oil 1 teaspoon hot pepper flakes 2 cups finely shredded green cabbage 1 cup finely shredded red cabbage
Intructions:  1. Wash and chop the cabbage and cut into 1 inch cubes.
2. Place the cabbage cubes and the red cabbage in a large bowl, add 1 tablespoon olive oil, and mix well.
3. Pour the mixture into the bowl, and add the cabbage, cabbage, and red cabbage to the bowl. Add the hot pepper flakes to the bowl, and stir well.
4. Add the shredded green cabbage, and mix well.
5. Place the cabbage cubes and the red cabbage in a large mixing bowl.
6. Mix the mixture well.
7. Pour the mixture into the bowl, and add the chopped red cabbage to the bowl.
8. Add the shredded cabbage and mix well.
9. Place the chopped red cabbage cubes and the chopped green cabbage in a large mixing bowl.
10. Pour the mixture into the bowl, and add the chopped red cabbage.
12. Place the chopped red cabbage cubes and the chopped green cabbage in a large mixing bowl.
13. Pour the mixture into the bowl, and add the chopped red cabbage.
14. Place the chopped red cabbage cubes and the chopped green cabbage in a large mixing bowl.
15. Pour the mixture into the bow
===========================

2 cups cooked spinach 1 cup ricotta cheese 2 eggs 1/2 cup sun-dried tomato, chopped (optional) 1/4 cup shredded fresh basil leaves or 1 tablespoon dried basil 1 tablespoon olive oil salt and pepper 1 partially 9-inch baked pie crust
Intructions:  1. Preheat oven to 350 degrees F.  2. Cut spinach into 2-inch cubes.  3. Preheat oven to 350 degrees F.  4. Preheat oven to 350 degrees F.  5. Place spinach in a bowl of a food processor.  6. In a bowl of food processor, pulse spinach until it resembles finely chopped tomatoes.  7. Add basil, tomato, and salt.  8. Blend until smooth.
9. Add the rest of the ingredients to the food processor.  10. Add the rest of the ingredients to the food processor and pulse until smooth.
11. Pour the mixture into a bowl of food processor and add the rest of the ingredients to the food processor.  12. Blend until smooth.
13. Place the pie crust on a baking sheet.  14. Bake for 15 minutes or until the crust has set.  15. Let cool completely.
16. Serve with a side of fresh basil and fresh basil leaves.
17. Enjoy!
You can make this recipe in any size pie crust recipe.  You can make this recipe in any size pie crust recipe.  You can also use any recip
===========================

2 cups shredded monterey jack cheese 1 (8 ounce) package cream cheese, softened 1 cup sour cream 1/2 cup mayonnaise 1/4 cup minced red bell pepper 1/4 cup minced green onion 2 tablespoons chopped fresh parsley 2 tablespoons fresh lemon juice 1 1/2 teaspoons Old Bay Seasoning 1/4 teaspoon ground red pepper 2 (8 ounce) containers lump crabmeat, picked 1/4 cup freshly grated parmesan cheese toasted french bread rounds or cracker
Intructions:  Mix all ingredients except the cheese and season to taste, and place in refrigerator.
1. Preheat oven to 400 degrees F.
2. In a large bowl, whisk together the flour, salt, and sugar. Add the remaining ingredients to the wet ingredients and stir until combined.
3. Using your hands and a spatula, gently mix the mixture into the flour mixture.
4. Pour into prepared baking dish and cover with foil.
5. Bake for 20 minutes or until golden brown and the edges have been lightly browned. Remove to wire rack to cool.
6. Serve with fresh parsley, green onions, or a drizzle of lemon juice.
Enjoy!
If you enjoyed this recipe, check out my new recipe for the "Pineapple Pie" and "Pineapple Pie" by Toni.
===========================

1 tablespoon butter (softened) 1 cup milk 1/4 cup heavy cream 2/3 cup flour 3 eggs 1/4 cup sugar 2 teaspoons vanilla 1 teaspoon lemon zest 1/2 teaspoon cinnamon 1/4 teaspoon salt 4 cups apples (tart-sliced) 1 tablespoon powdered sugar
Intructions:  Preheat oven to 350 degrees F.
Mix all ingredients together in a bowl. In a bowl, whisk together flour, milk and eggs.
In a separate bowl, whisk together butter, sugar, vanilla and lemon zest until well blended.
In a small bowl, whisk together the egg mixture and vanilla.
Add the flour mixture to the flour mixture.
Spread the mixture evenly over all sides (about 2 inches apart).
Spread the mixture evenly over the top of the cake.
Place the cake on top of the cake.
Roll out the sides of the cake, then place it back on top of the cake.
Place on a baking sheet and bake for about 30 minutes or until the edges are golden brown and the edges are golden brown and the cake is golden brown. Let cool for 5 minutes.
Once the cake is cool, remove the cake from the baking sheet and let cool for 5 minutes.
If you want to use a cookie cutter, you can use a toothpick to cut out the center of the cookie cutter.
To make the cookie cutter, you can use a sharp knife to cut the edges of the center.
Place the cookie cutter in the oven.
Remove the cookie cutter from the oven and let coo
===========================

6 quarts figs baking soda (one cup) 1 gallon water, boiling cold water 1 lemon, sliced (optional) 1 orange, sliced (optional) lemon juice (use the bottled kind) Syrup 2 1/4 cups sugar 5 1/4 cups water
Intructions:  1 1/2 cups chopped fresh basil 1/2 cup chopped fresh parsley 1/2 cup chopped fresh parsley leaves 1/4 cup chopped fresh basil leaves 1/2 teaspoon salt 2 cups chopped fresh basil leaves 1 cup chopped fresh basil leaves 1/2 cup chopped fresh basil leaves 1 teaspoon salt 3 cups chopped fresh basil leaves 1 cup chopped fresh basil leaves 1 teaspoon salt 1 teaspoon ground black pepper 2 tablespoons extra virgin olive oil 1 tablespoon chopped garlic powder 1/4 teaspoon ground cumin 1/4 teaspoon ground black pepper 1 tablespoon chopped cilantro 1/4 cup chopped cilantro 2 cups chopped fresh cilantro 1 cup chopped fresh cilantro 2 cups chopped green onions 1/4 cup chopped fresh green onion 1/4 cup chopped fresh green onion 1 cup chopped fresh green onion 1 cup chopped fresh green onion 1 cup chopped fresh green onion 1 1/2 cups chopped fresh cilantro 2 cups chopped fresh cilantro 2 tablespoons chopped cilantro 1 teaspoon ground cumin 2 tablespoons extra virgin olive oil 1 tablespoon chopped garlic powder 1/4 teaspoon ground cumin
Directions:
1. In large bowl, whisk together 1 cup chopped fresh basil, 1 teaspoon salt, 1 teaspoon ground black pepper, 1/4 teaspoon ground cumin, 1 teaspoon garlic powder,
===========================

1 cup vegetable stock 3 cups chopped broccoli (about 1/2 large bunch) 1 garlic clove, minced 2 teaspoons grated gingerroot 1 cup soymilk or 1 cup skim milk 1 teaspoon low sodium soy sauce 1/4 cup finely chopped red onion
Intructions:  Mix the ingredients together in a bowl.
Place the broccoli and garlic clove on a large plate and stir until smooth.
In a small saucepan, combine the milk, soy sauce, ginger, and onion and stir until combined.
Cover and let sit in the fridge for at least 30 minutes.
Once the broccoli is fully cooked, add the soymilk or skim milk and bring to the boil.
Once it's completely hot, add in the garlic, ginger, and onion.
Add in the soy sauce, red onion, and soy sauce and stir until smooth.
Remove from the heat and stir in the chopped red onion.
Add in the soy sauce and stir until smooth.
Add in the chopped red onion.
Add in the chopped red onion and stir until smooth.
Add in the chopped red onion and stir until smooth.
Bring to the boil. Reduce to a simmer and allow to cool to room temperature.
When the broccoli has cooled completely, remove from the heat.
Remove from the hot water and stir in the chopped red onion.
Add in the chopped red onion and stir until smooth.
Add in the chopped red onion and stir until smooth.
Add in the chopped red onion and stir until smoot
===========================

2 lbs chicken (bone-in or boneless, your choice) 1 cup Bourbon 1 cup soy sauce 1/2 cup brown sugar 1 teaspoon paprika 1 tablespoon molasses 3 garlic cloves, smashed 2 tablespoons ginger, freshly grated 1 1/2 teaspoons onion powder 1 1/2 teaspoons garlic powder
Intructions:  Preheat oven to 375 degrees F. Place chicken in a large mixing bowl and whisk in the soy sauce, brown sugar and paprika. Mix well. Pour into a greased 8 x 8 baking dish and sprinkle with chopped onion and garlic. Bake for 15-20 minutes. Remove from oven. Remove from oven and let cool.
Serves 8.
Ingredients
2 cups chicken broth
1/2 cup vegetable stock
1 tablespoon vegetable broth
1 teaspoon cornstarch
1 teaspoon salt
1/2 teaspoon pepper
2 cups water
1/2 cup vegetable oil
2 cups chicken stock, divided
1 tablespoon soy protein powder
1/2 teaspoon cumin
1 tablespoon chili powder
3 cloves garlic, minced
1/2 teaspoon ground ginger, minced
1/4 teaspoon cayenne pepper, to taste Directions:
1. In a large bowl, add water, cornstarch, salt and pepper and mix well. Add chicken stock and stir well. Add cornstarch and mix well. Add soy protein powder, cornstarch, and chili powder. Add chicken stock, stir well. Add cornstarch, salt and pepper. Mix well. Add chicken stock, stir well. Add soy protein powder, corns
===========================

FOR THE CHICKEN 1 (3 -3 1/2 lb) roasting chickens 2 ounces fresh gingerroot, crushed 2 garlic cloves, crushed 1 scallion, tie in a know 1 teaspoon salt 2 tablespoons vegetable oil or 2 tablespoons peanut oil chili sauce or soy sauce, for dipping FOR THE RICE 2 tablespoons vegetable oil or 2 tablespoons peanut oil 5 garlic cloves, finely chopped 5 shallots, finely chopped 1 1/2 cups long grain rice 3 3/4 cups chicken stock 1 teaspoon salt
Intructions:  Cook chicken in a large bowl. Add ginger root, shallots, garlic, and chili. Cook until the chicken is cooked through. Remove from heat and let cool on a rack. Place chicken back in the pot and cook until chicken is cooked through. Remove chicken from pot, cover, and let sit in a cool place for about 15 minutes. Remove from heat, and set aside.
Serve hot with a side of rice.
For the Chicken Salad:
Heat 1/4 cup of olive oil in a large saute pan over medium heat. Add chicken, shallots, ginger root, shallots, garlic, and chili, and cook until chicken is tender. Add chicken to a large saute pan over medium heat. Add ginger root, shallots, garlic, and chili, and cook until chicken is tender. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Add chicken to a large saute pan over medium heat. Cook for abou
===========================

1 cup honey, boiled and brought to room temperature 1/2 cup butter 1 cup brown sugar 4 eggs, separated 3 cups all-purpose flour 2 teaspoons baking soda 1/2 teaspoon baking powder 1 teaspoon cinnamon 1/4 teaspoon salt 1 cup sour cream 1 cup walnuts, chopped
Intructions:  1. Preheat oven to 400 degrees F, line 8-inch baking sheet with parchment paper and grease with oil.
2. In a medium bowl, whisk together butter and brown sugar. Stir in flour.
3. Add eggs, flour, baking soda and salt. Blend well, stirring constantly.
4. Fold in walnuts, walnuts and sour cream. Fold into the dough.
5. Bake for 15 minutes or until golden brown. Cool completely in a wire rack.
===========================

3 tablespoons brown sugar 3 tablespoons white vinegar 1 onion, peeled and cut into chunks 2 garlic cloves, peeled and lightly crushed 1 teaspoon peppercorn, lightly crushed 2 slices orange peel (about 2 inches each) 1 star anise 1 1/2 kg pickled pork
Intructions:  Preheat oven to 375 degrees.
Preheat the oven to 350 degrees. Place the pork on a lightly floured surface, and roast for 30 minutes. Drain the pork and discard any excess fat.
In a large bowl, mix together the vinegar, onion, garlic and pickles.
Add the onion mixture to the pork and stir to mix well.
In a small bowl, combine the onion mixture with the pickles. Add the pork to the pan, and cover with a lid.
Add the pork and cook for 30 seconds. Remove from the heat and let cool completely.
Serve immediately.
I hope I've helped you out!
This post may contain links to Amazon or other partners; your purchases via these links can benefit Serious Eats. Read more about our affiliate linking policy.
===========================

2 cups cooked rice 2 cups canned corn or 2 cups frozen corn 16 ounces joan of arc spicy hot chili beans, drained 1 (10 ounce) can rotel 4 ounces Velveeta reduced fat cheese product
Intructions:  1) Preheat oven to 350 degrees. In a large bowl, combine the onion, garlic, and chili peppers and mix until smooth.
2) In a small bowl, stir the cumin and cumin powder.
3) Pour the mixture into a large bowl and add the chopped onion, garlic and chili peppers. Mix well.
4) Mix well, cover, and bake for 15-20 minutes. Let cool completely before serving.
Enjoy!
This post is part of the series on the "The Power of Cooking" series.
===========================

Cake 1/2 cup cocoa powder, such as valrhona 1 cup warm water 1/2 cup plus 2 tablespoons vegetable oil 1 1/4 cups plus 2 tablespoons granulated sugar 2 large eggs 2 teaspoons vanilla extract 1 1/4 cups plus 2 tablespoons all-purpose flour 1 1/2 teaspoons baking soda 1 teaspoon kosher salt 3/4 cup plus 2 tablespoons buttermilk Caramel Sauce 1 cup granulated sugar 1 cup heavy cream 1 tablespoon salt Buttercream 8 whole egg whites 2 cups granulated sugar 2 lbs unsalted butter caramel sauce (from above) Eggnog Cheesecake 1 lb cream cheese 3/4 cup granulated sugar 2 large whole eggs plus 1 egg yolks 1/4 cup creme fraiche 1 teaspoon vanilla extract 1 tablespoon Irish whiskey, such as 2 gingers 2 teaspoons freshly grated nutmeg 1 teaspoon kosher salt Toffee Pecan Pie 9 inches pie crusts (homemade or store bought) 5 tablespoons unsalted butter 1 cup packed dark brown sugar 3 large eggs 2 teaspoons all-purpose flour 3/4 cup dark corn syrup 1 1/2 teaspoons kosher salt 1 tablespoon vanilla extract 1 1/2 toasted pecan pieces 3/4 cup chopped Heath candy bar, pieces Assembly 1/2 cup cocoa powder, such as valrhona 1 cup chopped Heath candy bar, pieces chocolate, santas chocolate gold coins, white sprinkles, gold leaf or marzipan, woodland creatures
Intructions:  1) Place the cake in the refrigerator for 1 hour. 2) Melt the chocolate. 3) In a large bowl, combine the cocoa powder, chocolate, salt, and nutmeg. 4) Add the eggs. 5) Add the flour and vanilla and mix well. 6) Pour into a large bowl. 7) Bake for 20 minutes or until the eggs are lightly golden brown. 8) Remove from the oven and let cool completely. (This is a good time to check the oven's temperature, because it will not be hot.) 9) Remove from the oven and allow to cool completely. 10) To assemble, use an electric mixer or stand mixer fitted with a paddle attachment to beat the eggs until light and fluffy. 11) In a medium bowl, combine the flour, butter, and salt. 12) Add the eggs, vanilla, and the flour mixture. 13) Beat the butter and sugar until well combined. 14) Slowly whisk in the egg mixture until smooth. 15) Add in the chocolate and mix until well combined. 16) Pour into the pie crust.
===========================

1 apple 1/4 cup cottage cheese, whipped 1 egg cinnamon and artificial sweetener, to taste
Intructions:  1/4 cup chopped fresh ginger, chopped 1/4 cup chopped fresh ginger, chopped 1/4 cup chopped fresh lime juice 1/2 cup chopped fresh basil, chopped 1/2 cup chopped fresh basil leaves 1/4 cup chopped fresh basil leaves 2 tablespoons chopped fresh lemon juice 1/4 cup chopped fresh lemon juice 1/2 cup chopped fresh lemon juice 1/2 cup chopped fresh lime juice 1/2 cup chopped fresh basil leaves 1/2 cup chopped fresh basil leaves 2 tablespoons chopped fresh basil leaves 1/3 cup chopped fresh basil leaves 1/3 cup chopped fresh basil leaves 1/4 cup chopped fresh basil leaves 1/2 cup chopped fresh basil leaves 3 tablespoons chopped fresh lemon juice 1/4 cup chopped fresh lemon juice 1/2 cup chopped fresh basil leaves 1 cup chopped fresh chopped fresh basil leaves 2 tablespoons chopped fresh basil leaves 1/4 cup chopped fresh basil leaves 1 cup chopped fresh basil leaves 1 cup chopped fresh basil leaves
Method:  1. Preheat oven to 350°F.
2. In a medium bowl, mix together the milk and the sugar.
3. In another bowl, mix together the eggs.
4. In another bowl, mix together the lemon juice and lime juice.
5. I
===========================

2 lbs carrots 2 medium onions 1 teaspoon pepper 1/2 teaspoon sugar 1/2 teaspoon Accent seasoning 8 slices bacon
Intructions:  1/2 cup chopped fresh cheddar cheese, chopped 1/2 cup chopped onion, sliced 1/2 cup chopped garlic 1/2 cup chopped cilantro, chopped 1/2 cup chopped cilantro leaves 1/4 cup chopped basil leaves 1/4 cup chopped cilantro leaves 1/4 cup chopped red pepper flakes 1/4 cup chopped cilantro leaves 2 teaspoons chili powder 2 tablespoons olive oil 1/3 cup chopped fresh cilantro, chopped 1/3 cup chopped garlic 1/3 cup chopped cilantro leaves 1/2 cup chopped cilantro leaves 1/2 cup chopped red pepper flakes 1/4 cup chopped cilantro leaves 2 tablespoons chili powder 2 cups diced tomatoes 2 cups diced tomatoes 1 cup diced tomatoes 1 cup diced tomatoes 2/3 cup diced tomatoes 1/2 cup diced tomatoes 1/4 cup diced tomatoes 1/2 cup diced tomatoes 1/2 cup diced tomatoes 1/2 cup diced tomatoes 1/2 cup diced tomatoes 1/2 cup diced tomato 1/2 cup diced tomatoes 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cup diced tomato 1/2 cu
===========================

DRESSING 1 tablespoon minced ginger 1 garlic clove, minced 2 teaspoons light soy sauce 1/4 cup rice wine vinegar 2 oranges, juice of 1 lime, juice of 1 teaspoon brown sugar SALAD 2 cups Chinese cabbage, shredded 100 g bean sprouts 1 red onion, thinly sliced 1 orange, peeled and sliced into 1 inch segments 2 grilled chicken breasts, sliced chopped fresh cilantro (for garnish) salt & freshly ground black pepper 1/4 cup fresh cilantro, chopped
Intructions:  Cook the cabbage, garlic, ginger, soy sauce, brown sugar, and salt in a large bowl over medium heat, stirring occasionally, until it begins to soften.
Add the rice wine vinegar and stir to combine and bring to a boil. Reduce heat to low and simmer for about 5 min. Remove from heat and allow to rest for 15 min. Remove from heat and set aside.
Heat the oil in a large skillet over medium heat. Add the onions, ginger and garlic and cook for 2 to 3 minutes or until the onion is translucent. Add the rice wine and saute until fragrant. Add the cooked rice wine and stir until fragrant. Stir in the chopped cilantro.
Add the chicken breasts and saute until tender. Add the cilantro, green onions and red pepper flakes and stir to combine.
Add the rice wine vinegar and stir to combine. Add the cilantro and stir to combine. Cook on high heat for 2 to 4 minutes or until the cilantro is tender and the rice wine vinegar is dissolved.
Serve with a salad, a side of a salad or a hot sauce.
===========================

1 lb ground meat 2 garlic cloves, minced 3 ounces shallots 1 (24 ounce) jar premium cheese flavor spaghetti sauce 1 (14 1/2 ounce) can Italian stewed tomatoes 1 tablespoon oregano 1 teaspoon salt
Intructions:  1. Preheat oven to 375 degrees F.
2. In a medium bowl, combine garlic, shallots and oregano until soft peaks form.
3. Stir in the pasta and season with salt and pepper.
4. Bake for about 10 minutes, until the pasta is lightly browned. Remove from oven and set aside.
5. Remove from oven and let cool.
6. Serve immediately, garnish with parsley and a side of parsley.
I hope this helps someone out. I love to make this recipe and I am always looking for new ways to use my kitchen equipment.
If you enjoyed this recipe or want to learn more about the process or learn how to make a homemade pasta dish, check out my blog here . If you are looking for recipes to make at home, check out my recipes here .
===========================

1/2 cup olive oil 2 ounces parmesan cheese 2 garlic cloves, peeled 3 tablespoons red wine vinegar 1 tablespoon mayonnaise 3 tablespoons sour cream 1/2 teaspoon italian seasoning 3 basil leaves, fresh salt, to taste pepper, to taste
Intructions:  Mix all ingredients except for the oil into a large bowl and mix in the parmesan cheese. Add the garlic and wine vinegar and stir well.
Add the parmesan cheese and stir well.
Mix the other ingredients. Add the basil leaves and sour cream and stir well.
Add the garlic and sour cream and stir well.
Add the other ingredients. Add the other spices and stir well.
The final product should be ready in about 15 minutes.
The last step is the finishing step.
Step 3: Add the ingredients to a large bowl and cover with a plastic wrap.
The finished product should be ready in about 30 minutes.
The final product should be ready in about 30 minutes.
Step 4: Pour the mixture into a large mixing bowl and add the parmesan cheese and the wine vinegar. Stir well.
The finished product should be ready in about 15 minutes.
Step 5: Add the rest of the ingredients to the bowl and stir well.
After about 15 minutes, the mixture should be ready to serve.
I hope this was helpful for you and your business!
===========================

1/4 cup whole almond, plus 2 tablespoons sliced almonds 1/4 teaspoon ground cinnamon 1/4 teaspoon freshly ground nutmeg 1 tablespoon trans-fat free margarine 2 cups sliced strawberries 1 cup blueberries 1 tablespoon granular sugar substitute 6 tablespoons part-skim ricotta cheese
Intructions:  Mix the almond and strawberries in a small bowl, and mix well. Add the whole almonds and ricotta cheese and mix until smooth, about 3 minutes. Place the bowl on a plate and cover with plastic wrap. Cover with plastic wrap and refrigerate for at least 2 hours.
I used a little more almond paste to make this, so it will be easier to work with. You can also use whole almonds if you don't want to use whole strawberries.
If you are using the whole strawberries, you can also use the whole blueberries.
For the dressing:  Mix the whole strawberry and blueberries in a bowl, then add the whole strawberries and mix. Mix well, until well combined.
For the filling:  Mix the whole strawberries and blueberries in a small bowl, then add the whole blueberries and mix. Mix well, until well combined.
For the filling, add 2 cups of the dressing and the remaining 2 cups of the dressing and mix well. Add the dressing mixture and mix well.
For the filling, add 1/3 Cup of the dressing and mix well. Mix well, until well combined.
To serve:  Add 1/2 Cup of the dressing and mix well.
For th
==========================="""

In [ ]:
gpt2_output_split = gpt2_output.split("\n===========================\n")
gpt2_output_slice = [item[item.find('Intructions:  ') + len('Intructions:  '):] for item in gpt2_output_split]
print(gpt2_output_slice[0])

my_model_output_split = my_model_output.split("\n===========================\n")
my_model_output_slice = [item[item.find('Intructions:  ') + len('Intructions:  '):] for item in my_model_output_split]
print(my_model_output_slice[0])

print(direc[0])

Mix together the egg whites, salt, and pepper in a bowl. Add the garlic and cook until the onion and tomato have softened, about 3 minutes. Add the tomatoes, the salt, and the parmesan. Cook for about 1 minute, stirring frequently. Stir in the lemon juice until it thickens. Stir in the parsley and the cumin. Add the tomatoes, and cook for about 2 minutes, stirring frequently. Add the lemon juice and cook until the parsley has softened. Stir in the parsley, the garlic, the parmesan, and the cumin. Add the tomatoes, and cook until the parsley has softened. Stir in the cumin. Add the garlic, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and cook until the cumin has softened. Add the tomatoes, and c

### Evaluate

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

gpt2_score = 0
my_model_score = 0
for i in range(20):
    reference = direc[i].split()
    gpt2_candidate = gpt2_output_slice[i].split()
    my_model_candidate = my_model_output_slice[i].split()
    gpt2_score += sentence_bleu([reference], gpt2_candidate)
    my_model_score += sentence_bleu([reference], my_model_candidate)

print("GPT2 BLEU Score:", gpt2_score/20)
print("my model BLEU Score:", my_model_score/20)

GPT2 BLEU Score: 0.00952901388420939
my model BLEU Score: 1.5808626786322325e-79


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
from evaluate import load

wer = load("wer")
gpt2_score = wer.compute(predictions=direc[:20], references=gpt2_output_slice)
my_model_score = wer.compute(predictions=direc[:20], references=my_model_output_slice)

print("GPT2 WER Score:", gpt2_score)
print("my model WER Score:", my_model_score)

GPT2 WER Score: 1.2624462200368778
my model WER Score: 2.844143272023233


### Human evaluate

In [ ]:
ingre = ["4 tomatoes", "1/2 c. evaporated milk", "1 onion", "2 Tbsp. butter", "pepper, chicken"]

ingre = [' '.join(text for text in ingre)][0]

test_idx = text_to_token_ids(ingre, tokenizer)

my_model_output = generate(
        model=my_model,
        idx=test_idx,
        max_new_tokens=256,
        context_size=BASE_CONFIG["seq_len"],
        top_k=1,
        temperature=0
    ).to('cpu')

print(remove_suffix(token_ids_to_text(my_model_output, tokenizer)))

4 tomatoes 1/2 c. evaporated milk 1 onion 2 Tbsp. butter pepper, chicken (whole) 1 c. milk 1/2 c. black pepper 1 small can chopped chilies 1 tsp. basil 1/2 tsp. basil 1/2 tsp. black pepper
Intructions: In large bowl bring 1 1/2 Mix well the oil, milk, egg, onion and 1/4 cup milk and stir until smooth. Add remaining ingredients to mol leaf; mix thoroughly. Pour into a lightly greased 11 x 7-inch pan and bake at 350° for 45 minutes or until golden brown.
